In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# import scispacy
import re
import tqdm
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
from openai import OpenAI
import hdbscan
import os
from dotenv import load_dotenv
import ast
import sqlite3

In [13]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"))

### Summarization using LLMs

In [6]:
messages = pd.read_csv('messages.csv')  # Load your messages dataframe


In [7]:
msg = messages.iloc[1]['content']  # Example message for testing
msg

"1. What is your job title/department? Animal Care Specialist at Riverside County Animal Services.\n2. Who is the department supervisor? Robert Brown\n3. Brief description of accident: As I was exiting the dog's kennel he got excited and and started jumping up and biting at me.\n4. Size of the dog and location on body that you were bitten: Large sized dog and under my right upper arm.\n5. Prior injury to the affected body part? No\n6. Any medical conditions that require maintenance medication? No\n7. Have you missed any time from work as a result of this accident? Just 2 hours on the date of the accident (6/3/24) because I left earlier to see a doctor.\n\nThanks!\n-Sarah Cooper"

In [8]:
SYSTEM = "You extract a short SUBJECT (≤5 words) and a 1 sentence SUMMARY for insurance claim messages. Return JSON."
user = f"Message:\n\"\"\"\n{msg}\n\"\"\"\nRespond as: {{\"subject\":\"...\",\"summary\":\"...\"}}"


In [14]:
resp = client.chat.completions.create(
    model="gpt-4o-mini",  # or your available model
    messages=[{"role":"system","content":SYSTEM},{"role":"user","content":user}],
    # avoid temperature if your model complains; otherwise use 0–0.2 for stability
    max_tokens=120
)
data = json.loads(resp.choices[0].message.content)

In [15]:
data

{'subject': 'Dog Bite Incident',
 'summary': 'Sarah Cooper, an Animal Care Specialist, was bitten by a large dog under her right upper arm while exiting the kennel.'}

In [25]:
def summarize_and_extract_subject_and_summary(conversation, model="gpt-4o-mini"):
    """
    Summarizes a conversation and extracts a short subject line.
    
    Args:
        conversation (str): Full text of the conversation or message thread.
        model (str): OpenAI model to use.

    Returns:
        dict: { "subject": str, "summary": str }
    """
    prompt = f"""
    You are given a conversation between a claimant and adjuster.
    1. Provide a short subject line (max 8 words) summarizing the main topic. 
    2. Provide a concise summary (1 sentence) covering the key points.

    Provide in JSON format:
    {{
        "subject": "<short subject line>",
        "summary": "<concise summary>"
    }}

    Conversation:
    {conversation}
    """

    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    response_text = completion.choices[0].message.content.strip()

    # Optional: crude split if model outputs in Subject/Summary format
    if "subject" in response_text and "summary" in response_text:
        try:
            response_json = json.loads(response_text)
            subject = response_json.get("subject", "")
            summary = response_json.get("summary", "")
        except json.JSONDecodeError:
            # Fallback if JSON parsing fails
            parts = response_text.split("\n")
            subject = parts[0].replace("Subject:", "").strip()
            summary = parts[1].replace("Summary:", "").strip()
    else:
        # Fallback to manual extraction
        lines = response_text.split("\n")
        subject = lines[0].strip()
        summary = " ".join(line.strip() for line in lines[1:]).strip()

    return {"subject": subject, "summary": summary}

In [26]:
# Example usage:
conversation_text = """
Claimant: I sent the reimbursement form last week. Have you received it?
Adjuster: Yes, but it’s missing the mileage section.
Claimant: Okay, I’ll resend with the missing info.
"""
result = summarize_and_extract_subject_and_summary(conversation_text)
print(result)

{'subject': 'Reimbursement Form Missing Information', 'summary': 'The claimant needs to resend the reimbursement form due to a missing mileage section.'}


In [32]:
from tqdm import tqdm
tqdm.pandas()

In [35]:
messages_subset = messages.head(50).copy()
dict_subject_summary = messages_subset['content'].progress_apply(
    lambda x: summarize_and_extract_subject_and_summary(x, model="gpt-4o-mini")
)


100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [38]:
dict_subject_summary

0                {'subject': '```json', 'summary': '{'}
1                {'subject': '```json', 'summary': '{'}
2     {'subject': 'Thank You from Claims Supervisor'...
3     {'subject': 'Follow-up Appointment Confirmatio...
4     {'subject': 'Appointment Rescheduling Update',...
5     {'subject': 'Request for Rescheduled Appointme...
6     {'subject': 'Voicemail Follow-up', 'summary': ...
7     {'subject': 'Missing Paperwork from MediFirst ...
8     {'subject': 'Claimant's Agreement to Proceed',...
9     {'subject': 'Gratitude Expressed', 'summary': ...
10    {'subject': 'Workers Compensation Claim Follow...
11    {'subject': 'Delayed Response from Claimant', ...
12    {'subject': 'Seeking Second Opinion on Treatme...
13    {'subject': 'Workers Compensation 2nd Opinions...
14    {'subject': 'Request for Personal Call', 'summ...
15    {'subject': 'Employee Earnings Report Correcti...
16    {'subject': 'Physical Therapy Appointment Conf...
17               {'subject': '```json', 'summary

In [ ]:
def extract_subject_summary(row):
    if isinstance(row, dict):
        return row.get("subject", None), row.get("summary", None)
    if isinstance(row, str):
        try:
            # Safely parse string to dict
            d = ast.literal_eval(row)
            if isinstance(d, dict):
                return d.get("subject", None), d.get("summary", None)
        except Exception:
            return None, None
    return None, None



In [42]:
# Apply to dataframe subset
messages_subset[["subject_clean", "summary_clean"]] = dict_subject_summary.apply(lambda x: pd.Series(extract_subject_summary(x)))

In [43]:
messages_subset

,thread_id,timestamp,role,content,subject_clean,summary_clean
0,0,2024-06-07 14:47:59,supervisor,"Good morning, Ms. Cooper - when you have a few...",```json,{
1,0,2024-06-10 13:18:38,claimant,1. What is your job title/department? Animal C...,```json,{
2,0,2024-06-10 13:39:40,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,Thank You from Claims Supervisor,Emily Martinez expresses gratitude in her mess...
3,0,2024-07-01 19:03:40,supervisor,Hi Sarah - According to the documents from you...,Follow-up Appointment Confirmation,The adjuster is inquiring if the claimant atte...
4,0,2024-07-02 3:35:15,claimant,I was unable to make it to an appointment on 6...,Appointment Rescheduling Update,The claimant rescheduled their appointment to ...
5,0,2024-07-09 16:12:15,supervisor,"Hi Sarah, I hope you're doing well. Could you ...",Request for Rescheduled Appointment Paperwork,The claimant is requesting paperwork from a re...
6,1,2024-03-20 19:45:18,adjuster,I got your voicemail. Perfect glad all is good.,Voicemail Follow-up,The claimant acknowledges the adjuster's voice...
7,1,2024-03-27 12:10:21,adjuster,"Good morning, Michael, MediFirst Care didn't u...",Missing Paperwork from MediFirst Care,The claimant is inquiring about missing paperw...
8,1,2024-03-27 12:28:08,claimant,Yes ma'am I can,Claimant's Agreement to Proceed,The claimant confirms their willingness to pro...
9,1,2024-03-27 12:39:59,adjuster,Thank you so much!,Gratitude Expressed,The claimant expresses appreciation to the adj...


## For full dataset

In [44]:
dict_subject_summary = messages['content'].progress_apply(
    lambda x: summarize_and_extract_subject_and_summary(x, model="gpt-4o-mini")
)
messages[["subject", "summary"]] = dict_subject_summary.apply(lambda x: pd.Series(extract_subject_summary(x)))

100%|██████████| 3079/3079 [47:58<00:00,  1.07it/s] 


In [51]:
messages

,thread_id,timestamp,role,content,subject,summary
0,0,2024-06-07 14:47:59,supervisor,"Good morning, Ms. Cooper - when you have a few...",```json,{
1,0,2024-06-10 13:18:38,claimant,1. What is your job title/department? Animal C...,```json,{
2,0,2024-06-10 13:39:40,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,Claim Acknowledgment and Contact Information,Emily Martinez provides her contact details fo...
3,0,2024-07-01 19:03:40,supervisor,Hi Sarah - According to the documents from you...,Follow-Up Appointment Confirmation,The adjuster is inquiring if the claimant atte...
4,0,2024-07-02 3:35:15,claimant,I was unable to make it to an appointment on 6...,Appointment Rescheduling Update,The claimant rescheduled their missed appointm...
...,...,...,...,...,...,...
3074,946,2022-07-01 13:39:39,claimant,No thanks,Claimant Declines Offer,The claimant has decided not to accept the offer.
3075,947,2022-10-04 13:30:59,claimant,"Sarah, if you recall in the documentation ther...",```json,{
3076,948,2024-09-05 13:56:09,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",Paperwork Submission Confirmation,Ms. Smith confirms sending and signing the pap...
3077,948,2024-09-13 12:32:02,claimant,"Ms. Smith, I went back to work on September 3r...",Return to Work Update,"Ms. Smith returned to work on September 3rd, 2..."


In [56]:
messages = messages.reset_index().rename(columns={'index': 'id'})

In [61]:
messages = messages.rename(columns={'id': 'message_id'})

In [62]:
messages

,message_id,thread_id,timestamp,role,content,subject,summary
0,0,0,2024-06-07 14:47:59,supervisor,"Good morning, Ms. Cooper - when you have a few...",```json,{
1,1,0,2024-06-10 13:18:38,claimant,1. What is your job title/department? Animal C...,```json,{
2,2,0,2024-06-10 13:39:40,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,Claim Acknowledgment and Contact Information,Emily Martinez provides her contact details fo...
3,3,0,2024-07-01 19:03:40,supervisor,Hi Sarah - According to the documents from you...,Follow-Up Appointment Confirmation,The adjuster is inquiring if the claimant atte...
4,4,0,2024-07-02 3:35:15,claimant,I was unable to make it to an appointment on 6...,Appointment Rescheduling Update,The claimant rescheduled their missed appointm...
...,...,...,...,...,...,...,...
3074,3074,946,2022-07-01 13:39:39,claimant,No thanks,Claimant Declines Offer,The claimant has decided not to accept the offer.
3075,3075,947,2022-10-04 13:30:59,claimant,"Sarah, if you recall in the documentation ther...",```json,{
3076,3076,948,2024-09-05 13:56:09,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...",Paperwork Submission Confirmation,Ms. Smith confirms sending and signing the pap...
3077,3077,948,2024-09-13 12:32:02,claimant,"Ms. Smith, I went back to work on September 3r...",Return to Work Update,"Ms. Smith returned to work on September 3rd, 2..."


In [64]:
messages.to_csv('all_messages_with_subject_and_summary.csv')

## Insert into Database

In [47]:
## Add column
def add_column(db_path, table_name, column_name, column_type="TEXT"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")
    conn.commit()
    conn.close()

# Example:
add_column("claims.db", "messages", "subject", "TEXT")

In [68]:
add_column("claims.db", "messages", "timestamp", "TEXT")

In [72]:
def list_columns(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table_name})")
    columns = cur.fetchall()
    conn.close()
    return columns

cols = list_columns("claims.db", "messages")
for col in cols:
    print(col)

(0, 'message_id', 'TEXT', 0, None, 1)
(1, 'thread_id', 'TEXT', 0, None, 0)
(2, 'ts_iso', 'TEXT', 1, None, 0)
(3, 'ts_unix', 'INTEGER', 1, None, 0)
(4, 'role', 'TEXT', 0, None, 0)
(5, 'content', 'TEXT', 0, None, 0)
(6, 'intents_json', 'TEXT', 0, None, 0)
(7, 'intent_primary', 'TEXT', 0, None, 0)
(8, 'sentiment', 'REAL', 0, None, 0)
(9, 'language', 'TEXT', 0, None, 0)
(10, 'metadata_json', 'TEXT', 0, None, 0)
(11, 'source_file', 'TEXT', 0, None, 0)
(12, 'line_no', 'INTEGER', 0, None, 0)
(13, 'subject', 'TEXT', 0, None, 0)
(14, 'summary', 'TEXT', 0, None, 0)
(15, 'timestamp', 'TEXT', 0, None, 0)
(16, 'all_intents', 'TEXT', 0, None, 0)
(17, 'primary_intent', 'TEXT', 0, None, 0)
(18, 'primary_intents', 'TEXT', 0, None, 0)


In [48]:
## Update the column for matching rows

def ensure_column(db_path: str, table: str, column: str, coltype: str = "TEXT"):
    """Add a column if it doesn't exist."""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # check existing columns
    cols = [r[1] for r in cur.execute(f"PRAGMA table_info({table})").fetchall()]
    if column not in cols:
        cur.execute(f"ALTER TABLE {table} ADD COLUMN {column} {coltype}")
        conn.commit()
    conn.close()

def update_column_from_csv(
    csv_path: str,
    db_path: str,
    table_name: str,
    match_column: str,
    update_column: str,
    coltype: str = "TEXT",
    chunksize: int = 50_000,
):
    """
    Update `table_name.update_column` in SQLite from values in CSV,
    matching on `match_column`. CSV must contain both columns.
    """
    # 1) Ensure target column exists
    ensure_column(db_path, table_name, update_column, coltype)

    # 2) Stream CSV in chunks (handles large files)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        if match_column not in chunk.columns or update_column not in chunk.columns:
            raise ValueError(f"CSV must contain '{match_column}' and '{update_column}' columns.")

        # Drop rows lacking either value
        sub = chunk[[match_column, update_column]].dropna()

        # Prepare parameter tuples
        params = [(row[update_column], row[match_column]) for _, row in sub.iterrows()]
        if not params:
            continue

        cur.executemany(
            f"UPDATE {table_name} SET {update_column} = ? WHERE {match_column} = ?",
            params
        )
        conn.commit()

    conn.close()

In [65]:
update_column_from_csv(
    csv_path="all_messages_with_subject_and_summary.csv",
    db_path="claims.db",
    table_name="messages",
    match_column="message_id",         # or message_id, etc.
    update_column="subject",          # the column you’re adding/updating
    coltype="TEXT"
)

In [66]:
update_column_from_csv(
    csv_path="all_messages_with_subject_and_summary.csv",
    db_path="claims.db",
    table_name="messages",
    match_column="message_id",         # or message_id, etc.
    update_column="summary",          # the column you’re adding/updating
    coltype="TEXT"
)